In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import redis
from insightface.app import FaceAnalysis
from sklearn.metrics import pairwise
import matplotlib.pyplot as plt

# Redis Connection
hostname = 'redis-12955.crce179.ap-south-1-1.ec2.redns.redis-cloud.com'
portnumber = 12955
password = '0AXzSpM3wa8utdJhuvvpRF8Iwc4TNVT3'
r = redis.StrictRedis(host=hostname, port=portnumber, password=password)

# Retrieve Data from Redis
retrieved_dict = r.hgetall(name='academy:register')
retrieved_series = pd.Series(retrieved_dict)
retrieved_series = retrieved_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))
index = [x.decode() for x in retrieved_series.index]
retrieved_series.index = index
retrieved_df = retrieved_series.to_frame().reset_index()
retrieved_df.columns = ['name_role', 'facial_features']
retrieved_df[['Name', 'Role']] = retrieved_df['name_role'].str.split('@', expand=True)
retrieved_df.drop(columns=['name_role'], inplace=True)

# Load InsightFace Model
faceapp = FaceAnalysis(name='buffalo_sc', root='insightface_model', providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5)

def ml_search_algorithm(dataframe, feature_column, test_vector, name_role=['Name', 'Role'], thresh=0.5):
    dataframe = dataframe.copy()
    X_list = dataframe[feature_column].tolist()
    x = np.asarray(X_list)
    
    similar = pairwise.cosine_similarity(x, test_vector.reshape(1, -1))
    similar_arr = np.array(similar).flatten()
    dataframe['cosine'] = similar_arr
    
    data_filter = dataframe.query(f'cosine >= {thresh}')
    if len(data_filter) > 0:
        data_filter.reset_index(drop=True, inplace=True)
        argmax = data_filter['cosine'].argmax()
        person_name, person_role = data_filter.loc[argmax][name_role]
    else:
        person_name = 'Unknown'
        person_role = 'Unknown'
    
    return person_name, person_role

# Load Test Image
def recognize_faces(image_path, dataframe):
    test_image = cv2.imread(image_path)
    results = faceapp.get(test_image)
    test_copy = test_image.copy()
    
    for res in results:
        x1, y1, x2, y2 = res['bbox'].astype(int)
        embeddings = res['embedding']
        person_name, person_role = ml_search_algorithm(dataframe, 'facial_features', test_vector=embeddings)
        
        color = (0, 0, 255) if person_name == 'Unknown' else (0, 255, 0)
        cv2.rectangle(test_copy, (x1, y1), (x2, y2), color, 2)
        text_label = f"{person_name} ({person_role})" if person_name != 'Unknown' else 'Unknown'
        cv2.putText(test_copy, text_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    cv2.imshow('Face Recognition', test_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
# Run on a test image
image_path = 'test_images/group_test.jpeg'
recognize_faces(image_path, retrieved_df)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


2025-01-30 18:59:51.851 Python[86885:1237329] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-30 18:59:51.851 Python[86885:1237329] +[IMKInputSession subclass]: chose IMKInputSession_Modern
